In [ ]:
#LEVEL 5 - 1
# endpoints.txt  (format: "<name> <hostname> <path>")
# auth auth.example.com /health
# users users.example.com /health
# orders orders.example.com /health
# cdn cdn.example.com /health
# search search.example.com /health
# billing billing.example.com /health
# profile profile.example.com /health
# admin admin.example.com /health
# reports reports.example.com /health
# metrics metrics.example.com /health


# Assume the real world results are:
# auth -> 200
# users -> timeout
# orders -> 503
# cdn -> DNS failure
# search -> 301 (redirect to /healthz) [treat as OK for this exercise]
# billing -> 200
# profile -> 500
# admin -> SSL error
# reports -> 200
# metrics -> 200
#
# With threshold_fail_pct=30 (>= 30% => ALERT):
# failing endpoints = users, orders, cdn, profile, admin  => 5 failing out of 10 = 50%
#
# Expected output:
# ALERT: failing=5 total=10 fail_pct=50
# - users reason=timeout
# - orders reason=http_503
# - cdn reason=dns
# - profile reason=http_500
# - admin reason=ssl

In [ ]:
#USE THIS OF REVISION #NO SOLUTION FOR THIS BELOW.
#LEVEL 5 - 1

import sys
import urllib.request
import urllib.error
import socket
import ssl

def classify_failure(exc):
    # returns one of: "timeout", "dns", "ssl", "http_5xx", "http_4xx", "other"
    msg = str(exc).lower()
    if "timed out" in msg:
        return "timeout"
    if "name or service not known" in msg or "nodename nor servname provided" in msg:
        return "dns"
    if "ssl" in msg:
        return "ssl"
    return "other"

def check_one(hostname, path="/health", timeout="2"):
    """
    returns: (ok: bool, reason: str)
    reason: "ok", "timeout", "dns", "ssl", "http_XXX", "other"
    """
    url = "https://" + hostname + path
    try:
        resp = urllib.request.urlopen(url, timeout=timeout)
        code = resp.getcode()
        # treat 2xx as OK only
        if 200 <= code < 300:
            return True, "ok"
        return False, "http_" + str(code)
    except urllib.error.HTTPError as e:
        # HTTPError is also an exception but has .code
        if e.code >= 500:
            return False, "http_5xx"
        return False, "http_4xx"
    except Exception as e:
        return False, classify_failure(e)

def parse_targets(lines):
    targets = []
    for line in lines:
        line = line.strip()
        if not line:
            continue
        name, host, path = line.split()
        targets.append((name, host, path))
    return targets

def main(argv):
    if len(argv) != 2:
        raise ValueError("usage: netmon.py <endpoints_file> <threshold_fail_pct>")

    path = argv[0]
    threshold_pct = argv[1]

    with open(path) as f:
        targets = parse_targets(f)

    failing = []
    for name, host, p in targets:
        ok, reason = check_one(host, p)
        if not ok and reason != "http_4xx":
            failing.append((name, reason))

    total = len(targets)
    fail_pct = (len(failing) / total) * 100

    if fail_pct > threshold_pct:
        print(f"ALERT: failing={len(failing)} total={total} fail_pct={fail_pct}")
        for name, reason in failing:
            print(f"- {name} reason={reason}")
    else:
        print(f"OK: failing={len(failing)} total={total} fail_pct={fail_pct}")

if __name__ == "__main__":
    main(sys.argv[1:])

In [ ]:
#LEVEL 5 - 1 (MY SOLUTION)

# endpoints.txt  (format: "<name> <hostname> <path>")
# auth auth.example.com /health
# users users.example.com /health
# orders orders.example.com /health
# cdn cdn.example.com /health
# search search.example.com /health
# billing billing.example.com /health
# profile profile.example.com /health
# admin admin.example.com /health
# reports reports.example.com /health
# metrics metrics.example.com /health

# Assume the real world results are:
# auth -> 200
# users -> timeout
# orders -> 503
# cdn -> DNS failure
# search -> 301 (redirect to /healthz) [treat as OK for this exercise]
# billing -> 200
# profile -> 500
# admin -> SSL error
# reports -> 200
# metrics -> 200
#
# With threshold_fail_pct=30 (>= 30% => ALERT):
# failing endpoints = users, orders, cdn, profile, admin  => 5 failing out of 10 = 50%
#
# Expected output:
# ALERT: failing=5 total=10 fail_pct=50
# - users reason=timeout
# - orders reason=http_503
# - cdn reason=dns
# - profile reason=http_500
# - admin reason=ssl


import sys
import urllib.request
import urllib.error
import socket
import ssl

def classify_failure(exc):
    # returns one of: "timeout", "dns", "ssl", "http_5xx", "http_4xx", "other"
    msg = str(exc).lower()
    if "timed out" in msg:
        return "timeout"
    if "name or service not known" in msg or "nodename nor servname provided" in msg:
        return "dns"
    if "ssl" in msg:
        return "ssl"
    return "other"

#"auth.example.com","/heath"
def check_one(hostname, path="/health", timeout="2"):
    """
    returns: (ok: bool, reason: str)
    reason: "ok", "timeout", "dns", "ssl", "http_XXX", "other"
    """
    url = "https://" + hostname + path
    #"https://auth.example.com/heath"
    try:
        resp = urllib.request.urlopen(url, timeout=timeout)
        code = resp.getcode()
        # treat 2xx as OK only
        if 200 <= code < 300: #lets say 1iter: 200//301
            return True, "ok" #(True, "ok"),(False,"http_301"),(),()
        return False, "http_" + str(code)
    except urllib.error.HTTPError as e:
        # HTTPError is also an exception but has .code
        if e.code >= 500:
            return False, "http_5xx"
        return False, "http_4xx"
    except Exception as e:
        return False, classify_failure(e)

def parse_targets(lines):
    targets = []
    for line in lines:
        line = line.strip() #strip by spaces
        if not line and line.startswith("#"): #added startswith
            continue
        #auth auth.example.com /health
        name, host, path = line.split() #we should better do, line= line.split(), if len(line) !=3: raise ValueError, then write name, host, path = line 
        targets.append((name, host, path)) #[("auth","auth.example.com","/heath"),("users", users.example.com, /health),(orders,orders.example.com,/health),()]
    return targets

def main(argv):
    if len(argv) != 2: #we are expecting 2 args if not throw valueerror
        raise ValueError("usage: netmon.py <endpoints_file> <threshold_fail_pct>")

    path = argv[0] #"endpoints.txt"
    threshold_pct = int(argv[1]) #"8" assume its valkue is "8", we converted it to int, so its 8

    with open(path) as f: #here there are no permission option given, like r, r+, w
    #(PP1* try and except for FileNotFound)
        targets = parse_targets(f)
        #[("auth","auth.example.com","/heath"),("users", users.example.com, /health),(orders,orders.example.com,/health),()]

    failing = []
    for name, host, p in targets:
        ok, reason = check_one(host, p)
        if not ok and reason != "http_4xx":
            failing.append((name, reason)) #[("users","http_301"),(),()]

    total = len(targets) #10 #failing and total are ints
    fail_pct = (len(failing) / total) * 100 #5/10 * 100 = 50

    if fail_pct > threshold_pct: #50 > "8"
        print(f"ALERT: failing={len(failing)} total={total} fail_pct={fail_pct}")
        for name, reason in failing:
            print(f"- {name} reason={reason}")
    else:
        print(f"OK: failing={len(failing)} total={total} fail_pct={fail_pct}")
        
        #ALERT: failing=5 total=10 fail_pact=50
        # - users reason=http_501
        # - orders reason=http_501 etc print continues for rest of the values in failing list for all 10 values
        
        #for OK: results
        #OK: failing=5 total=10 fail_pct=50
        

if __name__ == "__main__":
    main(sys.argv[1:]) #leave first two and take rest of the args

#-----------------------------------
    

Good effort — you’re thinking about types and parsing, which is the right direction at this difficulty. But there are a few key mistakes (and one you introduced) that an interviewer would zoom in on.

## The big struggles showing up in this answer

### 1) You introduced a new parsing bug in `parse_targets`

You wrote:

```python
if not line and line.startswith("#"):
    continue
```

This condition can **never** correctly skip comment lines:

* If line is `"# auth ..."` then `not line` is False, so it won’t skip.
* If line is empty `""`, then `line.startswith("#")` is False.

✅ Correct logic is **OR**, not AND:

```python
if not line or line.startswith("#"):
    continue
```

This is a *logic bug you introduced*.

---

## What the interviewer expects you to find in the original buggy code

### 1) Major bug (datatype)

In `check_one`:

```python
def check_one(..., timeout="2"):
    resp = urllib.request.urlopen(url, timeout=timeout)
```

`timeout` should be a **number** (int/float), not a string `"2"`.
This can cause a `TypeError` or weird behavior depending on environment.

Also in `main` (original):

```python
threshold_pct = argv[1]   # string
if fail_pct > threshold_pct:
```

`fail_pct` is float, `threshold_pct` is str → **TypeError** in Python 3.

✅ You fixed `threshold_pct` conversion. You did *not* fix timeout.

**Fix:** make timeout numeric:

```python
def check_one(..., timeout=2):
```

(or cast: `timeout=float(timeout)` if it comes from argv)

---

### 2) Logic error (requirements mismatch)

Requirement says: threshold is **inclusive** (`>= 30% => ALERT`)
But the code uses:

```python
if fail_pct > threshold_pct:
```

Should be:

```python
if fail_pct >= threshold_pct:
```

You kept `>`.

---

### 3) Edge case that can break

`parse_targets` assumes exactly 3 tokens:

```python
name, host, path = line.split()
```

This breaks if:

* blank line (handled)
* comment line (not handled in original)
* extra whitespace / missing path / trailing comments

Also: `total = len(targets)` then divide by `total`. If file is empty (or all comments), `total == 0` → **ZeroDivisionError**.

---

## More subtle requirement miss (that matters)

The expected behavior says: **301 should be treated as OK**.
But original `check_one` treats only 2xx as OK:

```python
if 200 <= code < 300: ok
else fail http_301
```

So it would incorrectly count `search` as failing.

Fix (for this exercise):

```python
if 200 <= code < 400:
    return True, "ok"
```

---

## Your answer: quick score

### Rating: **6/10**

You’re improving on types (you cast threshold), but you:

* missed the timeout datatype issue,
* kept `>` instead of `>=`,
* introduced a bug in comment handling (`and` vs `or`),
* didn’t handle 301 requirement.

---

## “Best possible” short patch (what you should say in interview)

High-level fixes:

1. Cast inputs to numeric types (`threshold_pct`, `timeout`)
2. Treat 2xx/3xx as OK (per requirement)
3. Skip blank/comment lines, validate token count
4. Use `>=` for inclusive threshold
5. Guard divide-by-zero

Minimal code ideas (not full rewrite):

```python
# parse_targets
if not line or line.startswith("#"): continue
parts = line.split()
if len(parts) != 3: continue

# check_one
def check_one(..., timeout=2): ...
if 200 <= code < 400: return True, "ok"

# main
threshold_pct = float(argv[1])
if total == 0: print("OK: failing=0 total=0 fail_pct=0"); return
if fail_pct >= threshold_pct: ...
```

---

### One tip to fix your biggest recurring issue (types)

Whenever you see:

* `argv[...]` → **string**
* `.replace()` / `.split()` → **string**
* `check_output()` → **bytes unless text=True**
  Ask: “Do I need `int(...)` / `float(...)` here before comparing or doing math?”

---

If you want, paste your **final corrected version** of this netmon snippet in 10–15 lines (just the changed lines), and I’ll rate it again.
